# 네이버 매장정보 크롤링

### 네이버 매장 존재 여부 확인 크롤링에서 만든 파일을 사용합니다

문제 
1. 강남구 매장이지만, 네이버로 검색하는 경우 강남구가 아닌 다른 지역의 매장명이 검색됨
2. 데이터 반영이 빠른편이지만, 네이버에 매장이 없는 경우가 있음 (없는 매장 : 맥도날드, 교촌, 굽네) 
3. 검색하면 전혀 다른 업체가 나오는 경우가 있음 -> 네이버 업태구분명을 크롤링해와서 참고하여 반영해야 
4. 나라에서 제공하는 data를 사용하는 경우 인기도를 알기 어려움. 아닌가. -> 네이버 지도 api를 사용해서 점수를 부여할까? 
5. 자료 업데이트 시기 문제 - data를 다운로드 받아서 사용하는 경우, 매장 정보는 계속 바뀐는데 업데이트가 늦잖아... 매장 정보의 변동이 생기 사람이 계속 업로드를 해야 하잖아.. 

In [1]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import zipfile
import os

c:\Users\hare0\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [3]:
# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install())


# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
# import chromedriver_autoinstaller
# chromedriver_autoinstaller.install()
# driver = webdriver.Chrome()


# 주소 이동
url = 'https://map.naver.com/'
driver.get(url)
time.sleep(1)

In [4]:
# 데이터 불러오기
original_res_df = pd.read_excel('combined_file.xlsx')
res_df = original_res_df.copy()
res_df = res_df.loc[res_df['검색여부'] == '가능'].reset_index(drop=True)
res_df.drop('Unnamed: 0', axis=1, inplace=True)

res_df

,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),시도,지역구,상세주소,사업장명_원본,검색여부
0,영업/정상,서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",정식당,한식,202203.971845,444853.761458,서울특별시,강남구,"강남대로114길 18,",정식당,가능
1,영업/정상,서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",채선당 샤브보트,한식,202462.794327,443827.961945,서울특별시,강남구,"강남대로 378,",채선당 샤브보트 강남역점,가능
2,영업/정상,서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",수아당,한식,203470.848439,447369.579852,서울특별시,강남구,"압구정로 343,",수아당,가능
3,영업/정상,서울특별시 강남구 역삼동 769-1 삼환 아르누보,"서울특별시 강남구 역삼로 204, 삼환 아르누보 지하1층 B09,B10호 중 107...",알찬한끼,분식,203453.074369,443736.685792,서울특별시,강남구,"역삼로 204,",알찬한끼,가능
4,영업/정상,서울특별시 강남구 삼성동 155-11,"서울특별시 강남구 삼성로96길 11, 지상2층 (삼성동)",세계관,한식,204952.006872,445195.026116,서울특별시,강남구,"삼성로96길 11,",세계관,가능
...,...,...,...,...,...,...,...,...,...,...,...,...
8291,영업/정상,서울특별시 강남구 대치동 937 롯데백화점,"서울특별시 강남구 도곡로 401, 롯데백화점 지하1층 (대치동)",간코,일식,204669.543367,443873.621189,서울특별시,강남구,"도곡로 401,",간코 롯데백화점강남점,가능
8292,영업/정상,서울특별시 강남구 신사동 528-3 덕산빌딩,"서울특별시 강남구 압구정로 108, 덕산빌딩 지상1층 10호 (신사동)",오토김밥,분식,201749.456414,446742.828124,서울특별시,강남구,"압구정로 108,",오토김밥 신사점,가능
8293,영업/정상,서울특별시 강남구 신사동 635-14 금광아크존1,"서울특별시 강남구 압구정로42길 36, 지상1층 101호 (신사동, 금광아크존1)",마들마들,한식,202990.870783,447146.555568,서울특별시,강남구,"압구정로42길 36,",마들마들,가능
8294,영업/정상,서울특별시 강남구 신사동 629-1,"서울특별시 강남구 언주로151길 21, 지상1층 (신사동)",세바,경양식,202731.421127,446609.460134,서울특별시,강남구,"언주로151길 21,",세바(Sevva),가능


In [7]:
# test용 5개
res_df = res_df[:6]
res_df

,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),시도,지역구,상세주소,사업장명_원본,검색여부
0,영업/정상,서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",정식당,한식,202203.971845,444853.761458,서울특별시,강남구,"강남대로114길 18,",정식당,가능
1,영업/정상,서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",채선당 샤브보트,한식,202462.794327,443827.961945,서울특별시,강남구,"강남대로 378,",채선당 샤브보트 강남역점,가능
2,영업/정상,서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",수아당,한식,203470.848439,447369.579852,서울특별시,강남구,"압구정로 343,",수아당,가능
3,영업/정상,서울특별시 강남구 역삼동 769-1 삼환 아르누보,"서울특별시 강남구 역삼로 204, 삼환 아르누보 지하1층 B09,B10호 중 107...",알찬한끼,분식,203453.074369,443736.685792,서울특별시,강남구,"역삼로 204,",알찬한끼,가능
4,영업/정상,서울특별시 강남구 삼성동 155-11,"서울특별시 강남구 삼성로96길 11, 지상2층 (삼성동)",세계관,한식,204952.006872,445195.026116,서울특별시,강남구,"삼성로96길 11,",세계관,가능
5,영업/정상,서울특별시 강남구 대치동 907-12 아티스톤,"서울특별시 강남구 역삼로 409, 지상1층 102호 (대치동, 아티스톤)",분식을 품다,분식,204506.415174,444321.238711,서울특별시,강남구,"역삼로 409,",분식을 품다,가능


In [11]:
res_info_df = pd.DataFrame()
restaurant_name_list = []
category_name_list = []
address_list = []
menu_list = []
price_list = []


In [12]:
# 일단 검색결과 안나오고 바로 페이지가 넘어간다고 가정하고 

for i in range(0,len(res_df)):
    # 검색어를 네이버 url에 포함시켜 이동 + 검색 
    
    name = res_df['상세주소'][i] + ' ' + res_df['사업장명'][i]
    driver.get('https://map.naver.com/p/search/{}'.format(name))

        
    time.sleep(3)  

    ## 화면이 안넘어가고 검색결과에 머물러 있을 때는 entryIframe 이 없음
    ## 그래서 검색어 쳤을때 entryIframe 있으면 --> 화면이 넘어갔다는 뜻이고 
    ## 없으면 아직 안넘어간 뜻이니깐 searchIframe 에서 클릭해서 넘어가야함 .
    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
            
            time.sleep(3) 
            # 가게 이름, 변수로 지정
            restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
            category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
            address = driver.find_element(By.CLASS_NAME,'LDgIH').text
            if driver.find_elements(By.CLASS_NAME,'ihmWt') :
                menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
                price = driver.find_element(By.CLASS_NAME,'mkBm3').text
            elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
                menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
                price = driver.find_element(By.CLASS_NAME,'Yrsei').text
            else : 
                menu = '메뉴없음'
                price = "가격없음"
            
        else : 
            # searchIframe 프레임 

                searchIframe = driver.find_element(By.ID,'searchIframe')
                driver.switch_to.frame(searchIframe)


                ## 클릭 
                driver.find_element(By.CLASS_NAME,'YwYLL').click()
                
                # 기본프레임 
                time.sleep(1)
                driver.switch_to.default_content()
            

                # entryIframe 프레임 
                entryIframe = driver.find_element(By.ID,'entryIframe')
                driver.switch_to.frame(entryIframe)

                time.sleep(3) 
                # 가게 이름, 변수로 지정
                restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
                category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
                address = driver.find_element(By.CLASS_NAME,'LDgIH').text
                if driver.find_elements(By.CLASS_NAME,'ihmWt') :
                    menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
                    price = driver.find_element(By.CLASS_NAME,'mkBm3').text
                elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
                    menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
                    price = driver.find_element(By.CLASS_NAME,'Yrsei').text
                else : 
                    menu = '메뉴없음'
                    price = "가격없음"



    except Exception as e : 
        restaurant_name = name 
        category_name = '전처리필요'
        address = '전처리필요'
        menu = '메뉴없음'
        price = "가격없음"
        print(f"에러 메시지: {str(e)}")


    ## 리스트 ㄱ 
    restaurant_name_list.append(restaurant_name)
    category_name_list.append(category_name)
    address_list.append(address)
    menu_list.append(menu)
    price_list.append(price)

    
    print('식당 이름:',restaurant_name)
    print('식당 카테고리:',category_name)
    print('주소:',address)
    print('대표 메뉴:',menu)
    print('가격:',price)
    print('-----------------------------')
res_info_df = pd.DataFrame({'식당이름':restaurant_name_list ,
                             '업태구분' :category_name_list ,
                             '주소' :address_list ,
                             '메뉴': menu_list,
                             '가격':price_list,
                             })

    

NameError: name 'review_url' is not defined

['정식당', '채선당 샤브보트 강남역점', '수아당', '알찬한끼', '세계관', '분식을 품다', '식당 민홍',
       '제주그집', '만수정 강남점', '은서네 국내산 대패삼겹살', '냉동고집 삼성점', '리하베스트', '팡뮤제',
       '만소당', '꼬모 서울', '겐지(KENJI)', '논현136', '피양옥', "세븐어클락(7O'CLOCK)",

In [13]:
res_info_df.head()


,식당이름,업태구분,주소,메뉴,가격
0,냉동고집 삼성점,"육류,고기요리",서울 강남구 삼성로 517 1층 104호,냉동삼겹살,"15,000원"
1,리하베스트,"카페,디저트",서울 강남구 역삼로 220 1층,메뉴없음,가격없음
2,팡뮤제 베이커리,베이커리,경기 김포시 풍무로 128 웰라움퍼펙트시티 1층,아메리카노,"4,800원"
3,만소당,돈가스,서울 강남구 논현로159길 10 104호,돈까스,"13,000원"
4,꼬모 서울,"카페,디저트",서울 강남구 봉은사로74길 19 1층,이탈리안 티라미수,"6,500원"


In [ ]:
res_info_df.to_csv('매장정보결과.csv')

,식당이름,업태구분,주소,메뉴,가격
0,정식당,한식,서울 강남구 강남대로114길 18 . 1층 정식당,소불고기주물럭,"16,000원"
1,정식당,한식,서울 강남구 강남대로114길 18 . 1층 정식당,소불고기주물럭,"16,000원"
2,채선당 샤브보트 강남역점,샤브샤브,서울 강남구 강남대로 378 . 지상1층,소고기 샤브샤브,"12,900원"
3,수아당 갤러리아백화점 명품관 WEST점,김밥,서울 강남구 압구정로 343,유부월남쌈,"18,000원"
4,알찬한끼,종합분식,서울 강남구 역삼로 204 삼환아르누보 지하1층 107호,알찬도시락,"5,000원"
5,세계관 삼성본점,한식,서울 강남구 삼성로96길 11 2층,투뿔 한우 곱도리탕,"32,000원"
6,분식을품다 한티점,종합분식,서울 강남구 역삼로 409 1층 102호,쌀 떡볶이,"5,900원"
7,식당민홍,일식당,서울 강남구 논현로146길 11 2층,오마카세 코스,"122,000원"
8,제주그집,한식,서울 강남구 논현로8길 11 지상1층,냉동삼겹살(150g),"14,000원"
9,만수정 강남점,"장어,먹장어요리",서울 강남구 강남대로 628 비율라빌딩 1층,민물장어 1kg ( 650~680g),"90,000원"
